In [ ]:
import geemap, ee

ee.Initialize()

roi = geemap.shp_to_ee(r'D:/work/new/shp/fw.shp')
fw = roi.geometry()

In [ ]:
new = geemap.shp_to_ee(r'D:/work/new/point/new.shp')
shp = geemap.shp_to_ee(r'D:/work/gee/shp/dissolve.shp')
shp_ee = shp.geometry()
Map = geemap.Map()
# Map

In [ ]:
def EVI(img):
    nir = img.select("SR_B4")
    red = img.select("SR_B3")
    blue = img.select("SR_B1")
    evi = img.expression("2.5*(B4 - B3)/(B4 + 6*B3 - 7.5*B1 + 1)",{"B4": nir,"B3": red ,"B1": blue})
    evi = evi.rename('EVI')
    return evi

def NDVI(img):
    ndvi = img.normalizedDifference(['SR_B4', 'SR_B3']).rename('NDVI')
    return ndvi

In [ ]:
# CRU  https://developers.google.com/earth-engine/datasets/catalog/IDAHO_EPSCOR_TERRACLIMATE
# bands = ['pr','tem','SR_B1','SR_B2','SR_B3','SR_B4','SR_B5','LSWI','NDVI','EVI','GCVI','RVI','EVI2','SAVI','slope']
bands = ['pr','tem','NDVI','EVI','slope','elevation','SR_B3','SR_B4']
label = 'CID'

# 计算平均气温
def fun(image):
    return (image.select('tmmx').add(image.select('tmmn'))).multiply(0.5).rename('tem')

# 投影
def project(image):
    return image.reproject(ee.ImageCollection("IDAHO_EPSCOR/TERRACLIMATE").first().select(0).projection())

def sample(image):
    return image.resample('bilinear').reproject(**{'crs':'EPSG:32628','scale': 30.0})

# L457去云
def maskL457sr(image):
    # Bit 0 - Fill
    # Bit 1 - Dilated Cloud
    # Bit 2 - Unused
    # Bit 3 - Cloud
    # Bit 4 - Cloud Shadow
    qaMask = image.select('QA_PIXEL').bitwiseAnd(31).eq(0)
    saturationMask = image.select('QA_RADSAT').eq(0)

    # Apply the scaling factors to the appropriate bands.
    opticalBands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
    thermalBand = image.select('ST_B6').multiply(0.00341802).add(149.0)

    # Replace the original bands with the scaled ones and apply the masks.
    return image.addBands(opticalBands, None, True) \
      .addBands(thermalBand, None, True) \
      .updateMask(qaMask) \
      .updateMask(saturationMask).copyProperties(image, ["system:index","system:time_start"])

def maskL8sr(image):
    qaMask = image.select('QA_PIXEL').bitwiseAnd(int('11111',2)).eq(0);
    saturationMask = image.select('QA_RADSAT').eq(0);

#   // Apply the scaling factors to the appropriate bands.
    opticalBands = image.select('SR_B.').multiply(0.0000275).add(-0.2);
    thermalBands = image.select('ST_B.*').multiply(0.00341802).add(149.0);
    
    return image.addBands(opticalBands, None, True)\
                 .addBands(thermalBands, None, True)\
                 .updateMask(qaMask)\
                 .updateMask(saturationMask)\
                 .copyProperties(image, ["system:time_start",'system:id'])


In [ ]:
image2 = []
precipitation = ee.ImageCollection("IDAHO_EPSCOR/TERRACLIMATE")\
                .filterBounds(fw)\
                .filter(ee.Filter.calendarRange(2020,2020,'year'))\
                .select('pr')\
                .sum().clip(fw)
image2.append(project(precipitation))

temperature = ee.ImageCollection("IDAHO_EPSCOR/TERRACLIMATE")\
                .filterBounds(fw)\
                .filter(ee.Filter.calendarRange(2020,2020,'year'))\
                .select(['tmmn','tmmx'])\
                .map(fun).mean().multiply(0.1)\
                .clip(fw)
image2.append(project(temperature))

dataset = ee.Image('USGS/SRTMGL1_003').clip(fw)
elevation = dataset.select('elevation')
slope = ee.Terrain.slope(elevation).rename('slope')
image2.append(elevation)
image2.append(slope)

image2 = ee.Image(image2)

l8m = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2")\
            .filterBounds(fw)\
            .filter(ee.Filter.calendarRange(2020-5,2020+5,'year'))\
            .filterMetadata('CLOUD_COVER', 'less_than', 25)\
            .sort('CLOUD_COVER', False)\
            .map(maskL8sr)\
            .mean()
image2 = image2.addBands(l8m)

image2 = image2.addBands(EVI(image2))
image2 = image2.addBands(NDVI(image2))
# image2 = sample(image2)

In [ ]:
# 特征值的提取
training = image2.select(bands).sampleRegions(**{
    'collection':new,
    'scale':30,
    'geometries': True
})

#  random uniforms to the training dataset.
withRandom = training.randomColumn() #样本点随机的排列

# 我们想保留一些数据进行测试，以避免模型过度拟合
split = 0.8

trainingPartition = withRandom.filter(ee.Filter.lt('random', split))#筛选80%的样本作为训练样本
testingPartition = withRandom.filter(ee.Filter.gte('random', split))#筛选20%的样本作为测试样本

# 分类方法选择smileCart() randomForest() minimumDistance libsvm
classifier = ee.Classifier.smileRandomForest(100).train(trainingPartition,label,bands)


for year in range(1995,2015,5):
    
#     outfile = outpath+os.sep+'{}'.format(year)
#     if os.path.exists(outfile+'.tif'):
#         continue
    
    print('{}_'.format(year))

    image = []
    precipitation = ee.ImageCollection("IDAHO_EPSCOR/TERRACLIMATE")\
                    .filterBounds(shp_ee)\
                    .filter(ee.Filter.calendarRange(year,year,'year'))\
                    .select('pr')\
                    .sum().clip(shp_ee)
    image.append(project(precipitation))

    temperature = ee.ImageCollection("IDAHO_EPSCOR/TERRACLIMATE")\
                    .filterBounds(shp_ee)\
                    .filter(ee.Filter.calendarRange(year,year,'year'))\
                    .select(['tmmn','tmmx'])\
                    .map(fun).mean().multiply(0.1)\
                    .clip(shp_ee)
    image.append(project(temperature))
    
    dataset = ee.Image('USGS/SRTMGL1_003').clip(shp_ee)
    elevation = dataset.select('elevation')
    slope = ee.Terrain.slope(elevation).rename('slope')
    image.append(elevation)
    image.append(slope)
    
    image = ee.Image(image)
        
    if year<=2010:
        l5m = ee.ImageCollection("LANDSAT/LT05/C02/T1_L2")\
                    .filterBounds(shp_ee)\
                    .filter(ee.Filter.calendarRange(year-5,year+5,'year'))\
                    .filterMetadata('CLOUD_COVER', 'less_than', 25)\
                    .sort('CLOUD_COVER', False)\
                    .map(maskL457sr)\
                    .mean()
        image = image.addBands(l5m)
    else:
        l8m = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2")\
                    .filterBounds(shp_ee)\
                    .filter(ee.Filter.calendarRange(year-5,year+5,'year'))\
                    .filterMetadata('CLOUD_COVER', 'less_than', 25)\
                    .sort('CLOUD_COVER', False)\
                    .map(maskL8sr)\
                    .mean()
        image = image.addBands(l8m)
    
    image = image.addBands(NDVI(image))
    image = image.addBands(EVI(image))
#     image = sample(image)

    classified = image.select(bands).classify(classifier)
    classified = classified.where(image.select('NDVI').lte(0),0)

    # 运用测试样本分类，确定要进行函数运算的数据集以及函数
    test = testingPartition.classify(classifier)
    test_accuracy = test.errorMatrix(label, 'classification')

#     print('总体精度:',test_accuracy.accuracy().getInfo())

#     ee_export(shp_ee, classified, outfile, 1000)
    geemap.ee_export_image_to_drive(classified.int8(),description = '{}'.format(year),scale = 30,region = shp_ee,folder='xcf',maxPixels = 1e13)
#     break

In [ ]:
# elevationVis = {
#   'min': 0.0,
#   'max': 7.0,
#   'palette': ['3ae237', 'b5e22e', 'd6e21f', 'fff705', 'ffd611', 'ffb613', 'ff8b13']
# }
# Map.addLayer(image.select('NDVI'),elevationVis,'image')
# Map.addLayer(classified,elevationVis,'classified')

In [1]:
!earthengine task list

EE5VKSBJK5QHCZFEIVHV4GTS  Export.image  202110          READY      ---
ONTRB4AUIHWRULMHRGOLB3QK  Export.image  20219           READY      ---
YFHZJDHDAOMW2QI3OKM4TCLQ  Export.image  20218           READY      ---
F52FT5XKA3DWEDBTCJ6UPQYP  Export.image  20217           READY      ---
JHD2CAQP3T363PVEFJSOG2B5  Export.image  20216           READY      ---
QD3TVDEJPISRKJZ5FEO5C2JO  Export.image  20215           READY      ---
7H4DO6XOCFXMGZBWJPZJTKUD  Export.image  20224           READY      ---
HU6ZBKP2KU4KXPKPRSHTDDXM  Export.image  20223           RUNNING    ---
4DWP5NVNRFZQC2LUE7ODK3FI  Export.image  20222           COMPLETED  ---
S2Z2GXUPIW5AHJRYV6GLJUSC  Export.image  20221           COMPLETED  ---
MICJVVZ2GNWS3QDY3G2KLNP5  Export.image  20220           COMPLETED  ---
WOHUIOJWJCCI7XI25YX5HLWW  Export.image  202110          COMPLETED  ---
EGSVKK3C7Z7RZWRG4MG4EP4W  Export.image  20219           COMPLETED  ---
XWY36LXSYCHRSGRP7XZOWYO4  Export.image  20218           COMPLETED  ---
UFL4SD

C5GMD6RAXL3N2DQKLPCBD7K2  Export.image  EVI_22_mean     CANCELLED  Cancelled.
JVLEIKUOTICOLMOIHEGQ7JJQ  Export.image  PRCP_22_mean    CANCELLED  Cancelled.
